## 1. Input for optional constraints

In [36]:
media_limits = {
    'izervay': {
        'pde': {
            1: {'lower bound': 60, 'upper bound': 100},
            2: {'lower bound': 50, 'upper bound': 116}
        }
    },
    'grizzly': {
        'pde': {
            1: {'lower bound': 0, 'upper bound': 12},
            2: {'lower bound': 150, 'upper bound': 200}
            }
    }
 }
media_input_percentage = {
    'izervay': {
        'pde': {1: 0.5, 2: 0.2},
        'dtc ctv': {1: 0.3, 2: 0.1}
    },
    'grizzly': {
        'pde': {1: 1, 2: 2}
    }
}
frozen_medias_data = {
    'izervay': {
        'pde': {1: 200.0, 2: 300.0},
        'dtc ctv': {1: 200.0, 2: 200.0}
    },
    'grizzly': {
        'pde': {1: 0.0, 2: 50.0}
    }
}
brand_limit = {
    'izervay': {'lower bound': 0, 'upper bound': 1000},
    'grizzly': {'lower bound': 0, 'upper bound': 5000}
}


## 2. Optimization

In [39]:
from SpendOptimization import SpendOptimization

brand = 'all'

# Instantiate the SpendOptimization class
optimizer = SpendOptimization(
    budget=10000,  # Total budget available for allocation
    media_budget_limits=None,  # Optional: Absolute spend limits for each media channel
    media_budget_limits_pct=None,  # Optional: Spend limits as a percentage of prior year spending per media
    locked_media_allocations=None,  # Optional: Pre-allocated spend limits for media channels
    brand=brand,  # Brands to include in the optimization
    brand_budget_constraints=None  # Optional: Budget limits per brand
)

output = optimizer.run()

Current working directory: c:\Users\Kuan-ChihLee\OneDrive - Blend 360\Desktop\Git_Repo\react-dashboard\backend\alpha-version
Optimization terminated successfully    (Exit mode 0)
            Current function value: -161976.9592296347
            Iterations: 121
            Function evaluations: 8862
            Gradient evaluations: 121


## 3. Output

In [40]:
import pandas as pd

records = []
for brand, medias in output['output'].items():
    for media, periods in medias.items():
        for period, metrics in periods.items():
            row = {
                'brand': brand,
                'media': media,
                'period': period
            }
            row.update(metrics)
            records.append(row)

df = pd.DataFrame(records)
df

,brand,media,period,optimal_spending,incremental_dollar
0,izervay,pde,1,13.934053,215.125676
1,izervay,pde,2,13.934053,215.125676
2,izervay,pde,3,13.934053,215.125676
3,izervay,pde,4,13.934053,215.125676
4,izervay,pde,5,13.934053,215.125676
...,...,...,...,...,...
67,xtandi,dtc ctv,8,50.210953,886.057655
68,xtandi,dtc ctv,9,50.226134,886.151409
69,xtandi,dtc ctv,10,50.217752,886.099641
70,xtandi,dtc ctv,11,50.224656,886.142279


In [4]:
df.sum()

brand                 izervayizervayizervayizervayizervayizervayizer...
media                 pdepdepdepdepdepdepdepdepdepdepdepdedtc ctvdtc...
period                                                              468
optimal_spending                                              99.999999
incremental_dollar                                         28268.091676
dtype: object